In [5]:
import functools
import math
from typing import Dict, List, Optional

from absl import logging
import fiddle as fdl
import jax
from jax import numpy as jnp
from paxml import base_experiment
from paxml import experiment_registry
from paxml import seqio_input
from paxml import tasks_lib
from paxml import trainer_lib
from paxml.tasks.lm import model_params
from paxml.tasks.lm.params import lm_cloud
from praxis import base_hyperparams
from praxis import base_input
from praxis import base_layer
from praxis import layers
from praxis import optimizers
from praxis import pax_fiddle
from praxis import schedules
from praxis.layers import normalizations  # XD
from praxis.layers import transformers
import seqio
import t5.data
from t5.data import preprocessors as t5_preprocessors
from praxis import base_hyperparams
import tensorflow as tf
import numpy as np
import jax.numpy as jnp
from praxis import py_utils
from praxis import base_input
from jax.experimental import PartitionSpec as P
from jax.experimental.multihost_utils import host_local_array_to_global_array, global_array_to_host_local_array

import pickle
import numpy as np
import jax.numpy as jnp
import jax
import sys
import time
import os
import re
import argparse

import jax
import jax.numpy as jnp
import numpy as np
import orbax
from optax import MaskedNode
from flax.traverse_util import flatten_dict, unflatten_dict
from etils import epath

import pickle
import numpy as np
import jax.numpy as jnp
import jax
import tensorflow as tf

os.environ["JAX_PLATFORMS"] = "cpu"


from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("baichuan-inc/Baichuan-7B", trust_remote_code=True)

paxml_to_mesh_key_and_shape = {
 'params.lm.embedding_lookup.emb_var': 'wte',
 'params.lm.transformer.repeat.sub.x_layers_0.ff_layer.ffn_layer1.linear.w': 'w3',
 'params.lm.transformer.repeat.sub.x_layers_0.ff_layer.ffn_layer1_gate.linear.w': 'w1',
 'params.lm.transformer.repeat.sub.x_layers_0.ff_layer.ffn_layer2.linear.w': 'w2',
 'params.lm.transformer.repeat.sub.x_layers_0.self_attention.query.w': 'wq',
 'params.lm.transformer.repeat.sub.x_layers_0.self_attention.key.w': 'wk',
 'params.lm.transformer.repeat.sub.x_layers_0.self_attention.value.w': 'wv',
 'params.lm.transformer.repeat.sub.x_layers_0.self_attention.post.w': 'wo',
 'params.lm.transformer.repeat.sub.x_layers_0.layer_norm.scale': 'attention_norm',
 'params.lm.transformer.repeat.sub.x_layers_0.ff_layer.layer_norm.scale': 'ffn_norm',
 'params.lm.final_ln.scale': 'ln_f',
 'params.lm.softmax.logits_ffn.linear.w': 'lm_head'}

A new version of the following files was downloaded from https://huggingface.co/baichuan-inc/Baichuan-7B:
- tokenization_baichuan.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [2]:
def _compute_grad_norm(grads):
  """Computes total grad norm."""
  grad_norms_squared = jax.tree_map(lambda x: jnp.sum(x * x), grads)
  grad_norms_squared, _ = jax.tree_util.tree_flatten(grad_norms_squared)
  return jnp.sqrt(jnp.sum(jnp.stack(grad_norms_squared)))

In [6]:
def load_data(path, key):
    with jax.default_device(jax.devices("cpu")[0]):
        data = pickle.load(open(path, 'rb'))[key]
        join_flat_data = {'.'.join(k): v for k, v in flatten_dict(data).items()}
    return join_flat_data

import pickle


path = '/home/lishengping/projects/mesh_easy_jax/paxml_summary_updates_and_grads_11.pkl'
join_flat_paxml_grads = load_data(path, 'grads')
join_flat_paxml_updates = load_data(path, 'transformed_grads')


In [7]:
path = '/home/lishengping/projects/mesh_easy_jax/paxml_summary_scale_grads_11.pkl'
join_flat_paxml_scale_grads = load_data(path, 'scale_grads')

In [19]:
data = pickle.load(open(path, 'rb'))

data.keys()

dict_keys(['learning/applied_grad_norm_aggregate_scalar', 'learning/clipped_grad_norm_aggregate_scalar', 'learning/grad_scale_aggregate_scalar', 'learning/is_valid_step_aggregate_scalar', 'learning/lr_aggregate_scalar', 'learning/raw_grad_norm_aggregate_scalar', 'learning/var_norm_aggregate_scalar', 'lm/num_unpadded_tokens_scalar', 'lm/ratio_unpadded_tokens_scalar', 'lm/total_aux_loss_scalar', 'lm/total_aux_loss_weight_scalar', 'lm/transformer/repeat/sub/x_layers_0/xformer_input_abs_max_0_scalar', 'lm/transformer/repeat/sub/x_layers_0/xformer_input_abs_max_1_scalar', 'lm/transformer/repeat/sub/x_layers_0/xformer_input_mean_0_scalar', 'lm/transformer/repeat/sub/x_layers_0/xformer_input_mean_1_scalar', 'lm/transformer/repeat/sub/x_layers_0/xformer_input_std_0_scalar', 'lm/transformer/repeat/sub/x_layers_0/xformer_input_std_1_scalar', 'lr_aggregate_scalar', 'scale_grads', 'vars/params/lm/embedding_lookup/emb_var_scalar', 'vars/params/lm/final_ln/scale_scalar', 'vars/params/lm/softmax/logi

In [20]:
for k, v in data.items():
    if 'learning' in k:
        print(k, v)

learning/applied_grad_norm_aggregate_scalar 5.929588e-09
learning/clipped_grad_norm_aggregate_scalar 1.0000007
learning/grad_scale_aggregate_scalar 0.18647198
learning/is_valid_step_aggregate_scalar 1.0
learning/lr_aggregate_scalar 5.4747645e-13
learning/raw_grad_norm_aggregate_scalar 5.3627357
learning/var_norm_aggregate_scalar 628.04407


In [9]:
for k, v in join_flat_paxml_scale_grads.items():
    print(k, v.shape)

params.lm.embedding_lookup.emb_var (64000, 4096)
params.lm.final_ln.scale (4096,)
params.lm.softmax.logits_ffn.linear.w (4096, 64000)
params.lm.transformer.repeat.sub.x_layers_0.ff_layer.ffn_layer1.linear.w (2, 4096, 11008)
params.lm.transformer.repeat.sub.x_layers_0.ff_layer.ffn_layer1_gate.linear.w (2, 4096, 11008)
params.lm.transformer.repeat.sub.x_layers_0.ff_layer.ffn_layer2.linear.w (2, 11008, 4096)
params.lm.transformer.repeat.sub.x_layers_0.ff_layer.layer_norm.scale (2, 4096)
params.lm.transformer.repeat.sub.x_layers_0.layer_norm.scale (2, 4096)
params.lm.transformer.repeat.sub.x_layers_0.self_attention.key.w (2, 4096, 32, 128)
params.lm.transformer.repeat.sub.x_layers_0.self_attention.post.w (2, 4096, 32, 128)
params.lm.transformer.repeat.sub.x_layers_0.self_attention.query.w (2, 4096, 32, 128)
params.lm.transformer.repeat.sub.x_layers_0.self_attention.value.w (2, 4096, 32, 128)


In [11]:
scale_grad = join_flat_paxml_scale_grads['params.lm.transformer.repeat.sub.x_layers_0.self_attention.value.w']
print(scale_grad.shape)
scale_grad

(2, 4096, 32, 128)


Array([[[[ 3.57445424e-05,  9.81641133e-05, -7.07777508e-05, ...,
          -4.55253867e-05,  7.50457548e-05,  1.05277453e-04],
         [-4.41027187e-05, -1.19504140e-04, -5.86850683e-06, ...,
           1.60050186e-05,  1.38710166e-05, -5.69067342e-05],
         [ 5.79737352e-05, -4.35692164e-06, -6.22417417e-07, ...,
          -4.69480547e-05, -4.97933906e-05,  5.58397332e-05],
         ...,
         [-2.41853617e-04,  2.40075292e-06, -2.31183603e-05, ...,
          -5.19273926e-05,  7.00664168e-05, -9.03394393e-05],
         [ 5.86850692e-05,  3.87677137e-05,  2.56080293e-05, ...,
          -5.97520702e-05,  1.67163525e-05,  4.42805504e-05],
         [ 2.84533667e-06, -3.89455454e-05, -4.49918844e-05, ...,
          -2.43631948e-05,  8.14477607e-05, -7.04220802e-05]],

        [[-4.19687167e-05, -2.66750305e-07,  4.14352144e-05, ...,
           4.30357177e-05, -1.29818482e-05, -2.31183603e-05],
         [ 6.68654102e-05,  7.43344208e-05,  9.78084472e-07, ...,
          -4.26800507e

In [17]:
scale_grad / raw_grad

Array([[[[0.18647198, 0.18647197, 0.18647198, ..., 0.18647198,
          0.18647197, 0.18647198],
         [0.18647198, 0.18647198, 0.18647198, ..., 0.18647198,
          0.18647198, 0.18647198],
         [0.186472  , 0.18647198, 0.186472  , ..., 0.18647198,
          0.18647198, 0.18647198],
         ...,
         [0.18647198, 0.186472  , 0.18647198, ..., 0.18647198,
          0.18647198, 0.18647198],
         [0.18647198, 0.186472  , 0.18647198, ..., 0.18647198,
          0.18647197, 0.18647197],
         [0.18647198, 0.186472  , 0.18647198, ..., 0.18647198,
          0.18647198, 0.18647198]],

        [[0.18647198, 0.18647198, 0.18647198, ..., 0.18647198,
          0.18647198, 0.18647198],
         [0.18647197, 0.18647198, 0.18647198, ..., 0.186472  ,
          0.18647198, 0.18647198],
         [0.18647197, 0.18647198, 0.18647198, ..., 0.18647198,
          0.18647198, 0.18647198],
         ...,
         [0.186472  , 0.18647198, 0.18647197, ..., 0.18647198,
          0.18647198, 0.1

In [13]:
raw_grad = join_flat_paxml_grads['params.lm.transformer.repeat.sub.x_layers_0.self_attention.value.w']
print(raw_grad.shape)
raw_grad

(2, 4096, 32, 128)


Array([[[[ 1.91688538e-04,  5.26428223e-04, -3.79562378e-04, ...,
          -2.44140625e-04,  4.02450562e-04,  5.64575195e-04],
         [-2.36511230e-04, -6.40869141e-04, -3.14712524e-05, ...,
           8.58306885e-05,  7.43865967e-05, -3.05175781e-04],
         [ 3.10897827e-04, -2.33650208e-05, -3.33786011e-06, ...,
          -2.51770020e-04, -2.67028809e-04,  2.99453735e-04],
         ...,
         [-1.29699707e-03,  1.28746033e-05, -1.23977661e-04, ...,
          -2.78472900e-04,  3.75747681e-04, -4.84466553e-04],
         [ 3.14712524e-04,  2.07901001e-04,  1.37329102e-04, ...,
          -3.20434570e-04,  8.96453857e-05,  2.37464905e-04],
         [ 1.52587891e-05, -2.08854675e-04, -2.41279602e-04, ...,
          -1.30653381e-04,  4.36782837e-04, -3.77655029e-04]],

        [[-2.25067139e-04, -1.43051147e-06,  2.22206116e-04, ...,
           2.30789185e-04, -6.96182251e-05, -1.23977661e-04],
         [ 3.58581543e-04,  3.98635864e-04,  5.24520874e-06, ...,
          -2.28881836e

In [21]:
norm_grad = _compute_grad_norm(join_flat_paxml_grads)

In [22]:
norm_grad

Array(5.36274, dtype=float32)

In [ ]:
lp.optimizer = pax_fiddle.Config(
      optimizers.Adam,
      beta1=cls.ADAM_BETA1 if cls.ADAM_BETA1 else 0.9,
      beta2=cls.ADAM_BETA2 if cls.ADAM_BETA2 else 0.999,
      # weight_decay=cls.WEIGHT_DECAY if cls.WEIGHT_DECAY else 0.0, # lsp: DEPRECATION
      # l2_regularizer_weight=cls.WEIGHT_DECAY if cls.WEIGHT_DECAY else 0.0,
      decoupled_weight_decay=cls.WEIGHT_DECAY if cls.WEIGHT_DECAY else 0.0,
      epsilon=cls.ADAM_EPSILON if cls.ADAM_EPSILON else 1e-6,
      epsilon_root=cls.ADAM_EPSILON_ROOT if cls.ADAM_EPSILON_ROOT else 0.0,
      clip_gradient_norm_to_value=cls.CLIP_GRADIENT_NORM_TO_VALUE # lsp: 1.0
      if cls.CLIP_GRADIENT_NORM_TO_VALUE
      else 5.0,
      clip_threshold=cls.ADAM_CLIP_THRESHOLD
      if cls.ADAM_CLIP_THRESHOLD
      else 1.0,
      sharded_adam=True,
  )

In [14]:
import pickle
import numpy as np
import jax.numpy as jnp
import jax
import sys
import time
import os
import re
import argparse

import jax
import jax.numpy as jnp
import numpy as np
import orbax
from optax import MaskedNode
from flax.traverse_util import flatten_dict, unflatten_dict
from etils import epath

import pickle
import numpy as np
import jax.numpy as jnp
import jax
import tensorflow as tf

os.environ["JAX_PLATFORMS"] = "cpu"


from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("baichuan-inc/Baichuan-7B", trust_remote_code=True)

paxml_to_mesh_key_and_shape = {
 'params.lm.embedding_lookup.emb_var': 'wte',
 'params.lm.transformer.repeat.sub.x_layers_0.ff_layer.ffn_layer1.linear.w': 'w3',
 'params.lm.transformer.repeat.sub.x_layers_0.ff_layer.ffn_layer1_gate.linear.w': 'w1',
 'params.lm.transformer.repeat.sub.x_layers_0.ff_layer.ffn_layer2.linear.w': 'w2',
 'params.lm.transformer.repeat.sub.x_layers_0.self_attention.query.w': 'wq',
 'params.lm.transformer.repeat.sub.x_layers_0.self_attention.key.w': 'wk',
 'params.lm.transformer.repeat.sub.x_layers_0.self_attention.value.w': 'wv',
 'params.lm.transformer.repeat.sub.x_layers_0.self_attention.post.w': 'wo',
 'params.lm.transformer.repeat.sub.x_layers_0.layer_norm.scale': 'attention_norm',
 'params.lm.transformer.repeat.sub.x_layers_0.ff_layer.layer_norm.scale': 'ffn_norm',
 'params.lm.final_ln.scale': 'ln_f',
 'params.lm.softmax.logits_ffn.linear.w': 'lm_head'}

In [15]:
def load_data(path, key):
    with jax.default_device(jax.devices("cpu")[0]):
        data = pickle.load(open(path, 'rb'))[key]
        join_flat_data = {'.'.join(k): v for k, v in flatten_dict(data).items()}
    return join_flat_data

In [22]:
path = '/home/lishengping/projects/mesh_easy_jax/paxml_summary_updates_and_grads_11.pkl'
data = pickle.load(open(path, 'rb'))

In [28]:
data['learning/grad_scale_aggregate_scalar']
data['learning/clipped_grad_norm_aggregate_scalar']
data['learning/applied_grad_norm_aggregate_scalar']
data['learning/lr_aggregate_scalar']

Array(5.4747645e-13, dtype=float32)

In [16]:
path = '/home/lishengping/projects/mesh_easy_jax/paxml_summary_updates_and_grads_11.pkl'
join_flat_paxml_grads = load_data(path, 'grads')
join_flat_paxml_updates = load_data(path, 'transformed_grads')

In [19]:
path = '/home/lishengping/projects/mesh_easy_jax//paxml_summary_scale_grads_11.pkl'
join_flat_paxml_scale_grads = load_data(path, 'scale_grads')

In [4]:
path = '/home/lishengping/projects/mesh_easy_jax/mesh_summary_grads_12.pkl'
join_flat_mesh_grads = load_data(path, 'grads')
path = '/home/lishengping/projects/mesh_easy_jax/mesh_summary_updates_12.pkl'

join_flat_mesh_updates = load_data(path, 'updates')

In [13]:
# path = '/home/lishengping/projects/mesh_easy_jax/mesh_summary_grads_20.pkl'
# join_flat_mesh_grads = load_data(path, 'grads')

# path = '/home/lishengping/projects/mesh_easy_jax/mesh_summary_updates_20.pkl'
# join_flat_mesh_updates = load_data(path, 'updates')

# path = '/home/lishengping/projects/mesh_easy_jax/paxml_summary_grads_20.pkl'
# join_flat_paxml_grads = load_data(path, 'grads')

# path = '/home/lishengping/projects/mesh_easy_jax/paxml_summary_updates_20.pkl'
# join_flat_paxml_updates = load_data(path, 'transformed_grads')

In [16]:
def search_key(key):
    for k, v in paxml_to_mesh_key_and_shape.items():
        if v in key:
            return k
        
        
def compute(mesh_value, paxml_value):
    abs_div_mean = np.abs(mesh_value - paxml_value).mean()
    print(f'abs_div_mean: {abs_div_mean}')
    
    abs_paxml_mean = np.abs(paxml_value).mean()
    print(f'abs_paxml_mean: {abs_paxml_mean}')
    
    abs_mesh_mean = np.abs(mesh_value).mean()
    print(f'abs_mesh_mean: {abs_mesh_mean}')
    
    ratio = abs_div_mean / abs_paxml_mean
    print(f'ratio: {ratio}')

# key = 'params.transformer.h.0.attention.wv.kernel'
# key = 'params.transformer.h.0.attention.wo.kernel'
# key = 'params.lm_head.kernel'
# key = 'params.transformer.h.0.attention_norm.kernel'
# key = 'params.transformer.wte.embedding'

def print_value_div(mesh_data, paxml_data):
    for key, value in mesh_data.items():
        layer = re.findall('h\.(\d+)\.', key)
        if layer: 
            layer = layer[0]

        mesh = value

        paxml_key = search_key(key)
        paxml = paxml_data[paxml_key]
        if layer: paxml = paxml[int(layer[0])]
        if paxml.shape != mesh.shape:
            paxml = paxml.reshape(*mesh.shape)

        if 'wo' in key:
            mesh = mesh.transpose()
        print(f'mesh key: {key}')
        print(f'paxml key: {paxml_key}')
        compute(mesh, paxml)
        print('\n\n\n')


In [18]:
print_value_div(join_flat_mesh_updates, join_flat_paxml_updates)

mesh key: params.lm_head.kernel
paxml key: params.lm.softmax.logits_ffn.linear.w
abs_div_mean: 2.442319768647394e-08
abs_paxml_mean: 4.0613115626607355e-13
abs_mesh_mean: 2.4423595590405967e-08
ratio: 60136.234375




mesh key: params.transformer.h.0.attention.wk.kernel
paxml key: params.lm.transformer.repeat.sub.x_layers_0.self_attention.key.w
abs_div_mean: 1.796113657803744e-08
abs_paxml_mean: 3.173914581676429e-13
abs_mesh_mean: 1.7961141907107958e-08
ratio: 56589.85546875




mesh key: params.transformer.h.0.attention.wo.kernel
paxml key: params.lm.transformer.repeat.sub.x_layers_0.self_attention.post.w
abs_div_mean: 1.8964882997352106e-08
abs_paxml_mean: 3.357891493072779e-13
abs_mesh_mean: 1.896519918886952e-08
ratio: 56478.546875




mesh key: params.transformer.h.0.attention.wq.kernel
paxml key: params.lm.transformer.repeat.sub.x_layers_0.self_attention.query.w
abs_div_mean: 1.7860692480553553e-08
abs_paxml_mean: 3.1566063781948706e-13
abs_mesh_mean: 1.786069958598091e-08
ratio

In [20]:
paxml_key = 'params.lm.transformer.repeat.sub.x_layers_0.self_attention.value.w'
join_flat_paxml_grads[paxml_key][1].reshape(4096, -1)

Array([[ 1.23977661e-04, -5.38825989e-05, -1.56402588e-04, ...,
        -4.29153442e-06, -2.27928162e-04, -2.32696533e-04],
       [ 1.11579895e-04,  6.77108765e-05, -5.86509705e-05, ...,
         1.21593475e-05, -2.95639038e-04, -1.66893005e-04],
       [ 1.94549561e-04,  1.30653381e-04, -2.19345093e-05, ...,
        -8.34465027e-05, -2.15530396e-04,  1.07884407e-05],
       ...,
       [ 1.83105469e-04,  8.39233398e-05, -3.99351120e-06, ...,
        -4.62532043e-05,  7.72476196e-05,  3.99351120e-06],
       [-2.86102295e-04,  2.70843506e-04,  3.43322754e-04, ...,
         2.93254852e-05,  3.47137451e-04,  1.96456909e-04],
       [ 4.79221344e-05, -1.16348267e-04, -1.29699707e-04, ...,
        -1.71661377e-05, -2.78472900e-04, -1.17778778e-04]],      dtype=float32)

In [21]:
paxml_key = 'params.lm.transformer.repeat.sub.x_layers_0.self_attention.value.w'
join_flat_paxml_scale_grads[paxml_key][1].reshape(4096, -1)

Array([[ 2.3118360e-05, -1.0047595e-05, -2.9164701e-05, ...,
        -8.0025092e-07, -4.2502215e-05, -4.3391385e-05],
       [ 2.0806525e-05,  1.2626181e-05, -1.0936763e-05, ...,
         2.2673776e-06, -5.5128399e-05, -3.1120871e-05],
       [ 3.6278041e-05,  2.4363195e-05, -4.0901714e-06, ...,
        -1.5560436e-05, -4.0190382e-05,  2.0117420e-06],
       ...,
       [ 3.4144039e-05,  1.5649352e-05, -7.4467795e-07, ...,
        -8.6249265e-06,  1.4404517e-05,  7.4467795e-07],
       [-5.3350064e-05,  5.0504725e-05,  6.4020074e-05, ...,
         5.4683815e-06,  6.4731408e-05,  3.6633708e-05],
       [ 8.9361356e-06, -2.1695692e-05, -2.4185361e-05, ...,
        -3.2010037e-06, -5.1927393e-05, -2.1962442e-05]], dtype=float32)

In [6]:
mesh_key = 'params.transformer.h.0.attention.wv.kernel'
join_flat_mesh_grads[mesh_key]

Array([[ 1.9328239e-04,  5.1764539e-04, -3.7900155e-04, ...,
        -1.4112140e-04,  4.2456028e-04, -3.7503926e-04],
       [-2.1960339e-04, -4.8307415e-06,  2.2056226e-04, ...,
         9.6562115e-05,  8.1557082e-05,  4.1553736e-04],
       [-7.2328880e-06,  1.5034423e-04,  1.0243332e-04, ...,
        -2.6589027e-05, -1.8966251e-05,  1.8726145e-05],
       ...,
       [-2.9393592e-05, -6.1136034e-06,  2.0994071e-04, ...,
        -3.6315174e-05,  7.2296942e-05,  1.2619805e-04],
       [-1.0201818e-04,  1.4139096e-04,  2.8773682e-04, ...,
        -5.7661018e-05,  1.7714423e-04, -9.6707059e-05],
       [ 7.7491117e-05, -3.2183191e-06,  1.4687705e-04, ...,
        -1.1834341e-04,  4.8965518e-04, -1.4924211e-04]], dtype=float32)

In [5]:
mesh_key = 'params.transformer.h.0.attention.wv.kernel'
join_flat_mesh_updates[mesh_key]

Array([[-6.5079120e-09, -5.7973111e-09,  6.8538912e-09, ...,
         6.4495063e-09, -5.6801013e-09,  3.6232899e-09],
       [ 6.8556569e-09,  4.7293978e-09, -4.2271933e-09, ...,
         1.4967327e-10, -6.5958892e-09, -6.8555819e-09],
       [ 5.0966604e-09, -6.8472392e-09, -2.4138047e-09, ...,
         5.1096625e-09, -4.2651740e-09,  3.9235322e-09],
       ...,
       [-3.3787719e-09,  5.1055649e-09, -6.7947079e-09, ...,
         5.9618149e-10, -4.2026711e-09, -6.5318533e-09],
       [-2.0984312e-09, -6.8538646e-09, -4.3111825e-09, ...,
         6.1734076e-09, -6.7156378e-09, -5.1671306e-10],
       [-5.7341301e-09, -4.5443014e-09,  1.3732160e-09, ...,
         6.2192140e-09, -6.8452226e-09,  1.7364565e-09]], dtype=float32)

In [12]:
x = join_flat_paxml_updates[paxml_key][0].copy()

In [13]:
x

Array([[[-2.5682719e-13, -2.6371114e-13, -2.5919317e-13, ...,
         -2.6386207e-13, -2.6378617e-13,  2.6372849e-13],
        [-2.6267758e-13,  2.6331492e-13,  2.6281299e-13, ...,
          2.6378536e-13,  2.6354884e-13, -2.6367276e-13],
        [ 2.6326453e-13, -2.6322122e-13, -2.6341974e-13, ...,
         -2.6369835e-13,  2.6362056e-13,  2.6386179e-13],
        ...,
        [ 2.6340385e-13,  2.6342654e-13, -2.6239555e-13, ...,
         -2.6384727e-13,  2.6388380e-13,  2.6282511e-13],
        [ 2.6360256e-13,  2.6370987e-13, -2.6375763e-13, ...,
          2.6385288e-13, -2.6384095e-13,  2.6384160e-13],
        [-2.6368401e-13, -2.6330714e-13,  2.6333013e-13, ...,
         -2.6364357e-13,  2.6352916e-13, -2.6355738e-13]],

       [[-2.6104767e-13, -2.6281603e-13,  2.5900172e-13, ...,
          2.6384214e-13, -2.6255430e-13,  2.6366005e-13],
        [ 2.6120696e-13,  2.6234006e-13,  2.6219811e-13, ...,
         -2.6223178e-13,  2.6360335e-13,  2.6354816e-13],
        [ 2.6332774e-13, 

In [9]:
# paxml_key = 'params.lm.transformer.repeat.sub.x_layers_0.self_attention.value.w'
paxml_key = 'params.lm.transformer.repeat.sub.x_layers_0.self_attention.query.w'

join_flat_paxml_updates[paxml_key][0].reshape(4096, -1)

Array([[-2.5682719e-13, -2.6371114e-13, -2.5919317e-13, ...,
        -2.6364357e-13,  2.6352916e-13, -2.6355738e-13],
       [-2.6104767e-13, -2.6281603e-13,  2.5900172e-13, ...,
        -2.6221156e-13,  2.6368800e-13,  2.6378335e-13],
       [-2.6323534e-13, -2.6358717e-13,  2.5671560e-13, ...,
        -2.6365821e-13,  2.6261418e-13, -2.6352168e-13],
       ...,
       [ 2.6286823e-13, -2.6151420e-13,  2.6247472e-13, ...,
         2.6368908e-13,  2.6365452e-13,  2.6215927e-13],
       [ 2.6358790e-13, -2.6341180e-13, -2.6257162e-13, ...,
         2.6387700e-13,  2.6387594e-13,  2.6270541e-13],
       [-2.5898443e-13, -2.6360321e-13,  2.6024276e-13, ...,
        -2.6378747e-13, -2.6379552e-13, -2.6372483e-13]], dtype=float32)

In [36]:
paxml_key = 'params.lm.transformer.repeat.sub.x_layers_0.self_attention.value.w'
join_flat_paxml_grads[paxml_key][1].reshape(4096, -1)

Array([[ 1.39236450e-04,  6.77108765e-05, -2.00271606e-04, ...,
         1.26838684e-04, -7.96318054e-05, -6.86645508e-05],
       [ 1.01089478e-04,  1.71661377e-04, -1.02519989e-04, ...,
         2.03132629e-04, -9.25064087e-05, -2.36511230e-04],
       [ 1.59263611e-04,  1.21116638e-04, -1.04904175e-04, ...,
         9.72747803e-05, -1.64031982e-04, -2.63214111e-04],
       ...,
       [-4.26769257e-05,  1.13487244e-04,  1.12533569e-04, ...,
        -8.34465027e-05, -1.43051147e-06,  2.05039978e-04],
       [-2.47955322e-04, -1.46865845e-04,  2.63214111e-04, ...,
        -2.93731689e-04,  1.52587891e-04,  2.61306763e-04],
       [ 6.67572021e-05, -3.67164612e-05, -2.89678574e-05, ...,
         2.19345093e-05, -1.76429749e-04, -8.48770142e-05]],      dtype=float32)

In [41]:
paxml_key = 'params.lm.transformer.repeat.sub.x_layers_0.self_attention.value.w'
join_flat_paxml_updates[paxml_key][0].reshape(4096, -1)

Array([[-6.5132676e-13, -8.2958336e-13,  3.7946138e-13, ...,
         2.6585236e-13, -8.5747072e-13,  5.3261074e-13],
       [ 3.7791469e-13, -2.6369897e-13, -2.2857207e-15, ...,
        -5.0401095e-13,  4.6656862e-13, -1.8132358e-13],
       [-1.1992928e-13, -5.3921456e-13,  5.9534799e-13, ...,
        -2.8798806e-13, -5.0337436e-13,  7.1840776e-13],
       ...,
       [ 5.0955247e-13,  6.7409815e-13, -8.0526319e-13, ...,
        -1.9232666e-13,  3.4371922e-13, -4.7418238e-13],
       [-7.8440103e-14,  6.1712083e-13, -6.2715203e-13, ...,
        -2.1413658e-13,  3.1385192e-13,  5.5712363e-13],
       [-6.4435854e-13, -2.3589708e-13, -3.5760725e-13, ...,
         3.1307249e-14, -1.9757548e-13,  6.8405221e-13]], dtype=float32)

In [43]:
mesh_key = 'params.transformer.h.0.attention.wv.kernel'
join_flat_mesh_updates[mesh_key]

Array([[-3.6260488e-08, -4.6807770e-08,  2.1423556e-08, ...,
         1.5807831e-08, -4.8053469e-08,  2.9641097e-08],
       [ 2.1609006e-08, -1.5040072e-08, -2.7957311e-10, ...,
        -2.9796704e-08,  2.5277748e-08, -1.0959249e-08],
       [-6.3714385e-09, -3.0085367e-08,  3.3318813e-08, ...,
        -1.5373152e-08, -2.7388452e-08,  4.0757513e-08],
       ...,
       [ 2.9059981e-08,  3.7604494e-08, -4.5648978e-08, ...,
        -1.0360052e-08,  1.9210352e-08, -2.6716091e-08],
       [-3.7428425e-09,  3.4799367e-08, -3.5481126e-08, ...,
        -1.1262707e-08,  1.8317484e-08,  3.0930945e-08],
       [-3.6331844e-08, -1.3647225e-08, -2.0733587e-08, ...,
         2.3587405e-09, -1.0664245e-08,  3.7684423e-08]], dtype=float32)

In [42]:
mesh_key = 'params.transformer.h.0.attention.wv.kernel'
join_flat_mesh_grads[mesh_key]

Array([[-1.7257153e-04,  1.4990101e-04, -3.7721900e-05, ...,
         3.6485860e-04,  2.7983548e-04,  3.6086029e-04],
       [-6.1559171e-05, -1.3099423e-04, -1.1949951e-04, ...,
         8.5093794e-05, -1.1436395e-04, -3.6781724e-04],
       [-3.3753412e-04, -5.7417910e-05, -5.7470828e-04, ...,
         2.8695373e-04,  2.2696270e-04,  7.0971255e-05],
       ...,
       [ 9.6970442e-05, -2.4336045e-06,  3.9360576e-04, ...,
        -2.2549935e-05, -1.3949361e-04, -8.4047497e-05],
       [ 2.9162511e-05, -2.1473248e-04,  1.9319679e-04, ...,
        -7.8336729e-05, -3.3541754e-04, -3.0200215e-04],
       [-9.3519830e-06, -1.1854911e-04,  1.9308891e-04, ...,
         3.1782364e-04, -2.1051706e-04, -3.8374672e-04]], dtype=float32)

In [ ]:
print_value_div(join_flat_mesh_grads, join_flat_paxml_grads)

In [2]:
import pickle
import numpy as np
import jax.numpy as jnp
import jax
import sys
import time
import os
import re
import argparse

import jax
import jax.numpy as jnp
import numpy as np
import orbax
from optax import MaskedNode
from flax.traverse_util import flatten_dict, unflatten_dict
from etils import epath

from praxis import base_hyperparams
from praxis import pax_fiddle
from praxis import py_utils
from paxml import checkpoints
from paxml import checkpoint_managers
from paxml import train_states
from paxml import trainer_lib

import pickle
import numpy as np
import jax.numpy as jnp
import jax
import tensorflow as tf

os.environ["JAX_PLATFORMS"] = "cpu"


from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("baichuan-inc/Baichuan-7B", trust_remote_code=True)

TypeError: non-default argument 'user_summary_level' follows default argument

In [2]:
with jax.default_device(jax.devices("cpu")[0]):
    mesh_data = flatten_dict(pickle.load(open('/home/lishengping/projects/mesh_easy_jax/3_mesh_debug.pkl', 'rb')))
#     grad_data = flatten_dict(pickle.load(open('/home/lishengping/projects/mesh_easy_jax/mesh_grads.pkl', 'rb')))
    mesh_data = {'.'.join(k): v for k, v in mesh_data.items()}
# mesh_data.keys()    

In [39]:
mk

('params', 'transformer', 'wte', 'embedding')

In [40]:
pkey

'ln_f'

In [46]:
layer

0

In [3]:
mesh_data.keys()

dict_keys(['input_ids', 'last_output', 'lm_logits', 'loss', 'mask', 'mean_loss', 'transformer.h.0.attention.[dot]attn_weights', 'transformer.h.0.attention.[dot]attn_weights_mask', 'transformer.h.0.attention.[dot]attn_weights_softmax', 'transformer.h.0.attention.[dot]before_key', 'transformer.h.0.attention.[dot]before_query', 'transformer.h.0.attention.[dot]scale_query', 'transformer.h.0.attention.attention_maskNone', 'transformer.h.0.attention.attn_score_softmax', 'transformer.h.0.attention.score_dot_value', 'transformer.h.0.attention.value_post', 'transformer.h.0.attention.value_post_drop', 'transformer.h.0.attention.xk_projNone', 'transformer.h.0.attention.xk_rotaryNone', 'transformer.h.0.attention.xq_projNone', 'transformer.h.0.attention.xq_rotaryNone', 'transformer.h.0.attention.xv_projNone', 'transformer.h.0.attention_norm.x_cheng', 'transformer.h.0.attention_norm.x_eps', 'transformer.h.0.attention_norm.x_mean', 'transformer.h.0.attention_norm.x_rsqrt', 'transformer.h.0.attention_

In [43]:
layer

0

In [317]:
def search_mesh_key(key, layer=None):
    for k, v in mesh_data.items():
        if layer is None and key in k:
            return k
        else:
            if key in k and f'{layer}.' in k:
                return k
        
# mesh_key = 'input_embeds'
l = None
# mesh_key = 'input'
mesh_key = 'input_norm'
# mesh_key = 'attn_weights'
mesh_key = 'attn_weights_mask'
mesh_key = 'xv_proj'
mesh_key = 'attn_score_softmax'
mesh_key = 'score_dot_value'
mesh_key = 'value_post'
mesh_key = 'residual_attn_drop'
mesh_key = 'ffn_norm_'

# mesh_key = 'ffn1_gate_output'
# mesh_key = 'ffn1_gate_output_activ'
mesh_key = 'ffn1_output'
mesh_key = 'ffn1_output_activ'
mesh_key = 'ffn2_output'
mesh_key = 'residual_ffn_output'
# mesh_key = search_mesh_key(mesh_key, layer=l)

mesh_key = 'transformer.last_output' 
mesh_key = 'last_norm'
mesh_key = 'loss'

mesh_key = search_mesh_key(mesh_key)

l = 1
print(f'mesh_key: {mesh_key}')

if l is None:
    mesh = mesh_data[mesh_key].astype(jnp.float32)[1]
else:
    mesh = mesh_data[mesh_key].astype(jnp.float32)
print(f'mesh shape: {mesh.shape}')
mesh

mesh_key: loss
mesh shape: (8, 2047)


Array([[ 0.     ,  0.     ,  0.     , ..., -2.78125, -5.03125, -7.84375],
       [ 0.     ,  0.     ,  0.     , ...,  0.     ,  0.     ,  0.     ],
       [ 0.     ,  0.     ,  0.     , ...,  0.     ,  0.     ,  0.     ],
       ...,
       [ 0.     ,  0.     ,  0.     , ...,  0.     ,  0.     ,  0.     ],
       [ 0.     ,  0.     ,  0.     , ...,  0.     ,  0.     ,  0.     ],
       [ 0.     ,  0.     ,  0.     , ...,  0.     ,  0.     ,  0.     ]],      dtype=float32)

In [47]:
k

('params', 'lm_head', 'kernel')

In [49]:
paxml_key

'params.lm.softmax.logits_ffn.linear.w'

In [374]:
def search_paxml_key(key, layer=None, other=''):
    for k, v in paxml_data.items():
        if layer is None:
            if '[lsp]' + key in k and other in k:
                return k  
        else:
            if '[lsp]' + key in k and other in k and f'_{layer}' in k:
                return k  
            
paxml_key = 'inputs_drop' 
# paxml_key = 'inputs_sc'
paxml_key = 'inputs_normalized'
paxml_key = 'padded_logits'
paxml_key = 'before_value'
paxml_key = 'probs'
paxml_key = 'encoded'
paxml_key = 'encoded_post'
paxml_key = 'ffn_input' # residual_attn_drop

paxml_key = 'linear_input' # other = 'gate'  -> ffn_norm_
paxml_key = 'linear_proj' # other = 'gate'
paxml_key = 'linear_proj_activ' # other = 'gate'
# paxml_key = search_paxml_key(paxml_key, layer=1, other='gate')

# paxml_key = 'linear_proj' # other = 'ffn_layer1'
# paxml_key = search_paxml_key(paxml_key, layer=1, other='ffn_layer1')

paxml_key = 'ffn1_activations'
paxml_key = 'ffn2_outputs'
paxml_key = 'ffn_outputs' # -> residual_ffn_output
paxml_key = search_paxml_key(paxml_key, layer=1)
paxml_key = 'last_output_s'
paxml_key = 'last_output_norm'
paxml_key = 'per_token_xent'
paxml_key = 'total_aux_loss_weight_'
paxml_key = search_paxml_key(paxml_key, layer=None)

print(f'paxml_key: {paxml_key}')
paxml = paxml_data[paxml_key].astype(jnp.float32)
if len(paxml.shape) > 2:
    paxml = paxml[1]
print(f'paxml shape: {paxml.shape}\n')
paxml

paxml_key: None


KeyError: "None; available attributes: ['learning/applied_grad_norm_aggregate_scalar', 'learning/clipped_grad_norm_aggregate_scalar', 'learning/grad_scale_aggregate_scalar', 'learning/is_valid_step_aggregate_scalar', 'learning/lr_aggregate_scalar', 'learning/raw_grad_norm_aggregate_scalar', 'learning/var_norm_aggregate_scalar', 'lm/[lsp]input_ids_scalar', 'lm/[lsp]inputs_drop_scalar', 'lm/[lsp]inputs_scalar', 'lm/[lsp]last_output_norm_scalar', 'lm/[lsp]last_output_scalar', 'lm/[lsp]per_token_xent_scalar', 'lm/final_ln/[lsp]x_cheng_scalar', 'lm/final_ln/[lsp]x_eps_scalar', 'lm/final_ln/[lsp]x_mean_scalar', 'lm/final_ln/[lsp]x_rsqrt_scalar', 'lm/final_ln/[lsp]x_square_scalar', 'lm/num_unpadded_tokens_scalar', 'lm/ratio_unpadded_tokens_scalar', 'lm/softmax/logits_ffn/[lsp]linear_input_scalar', 'lm/softmax/logits_ffn/[lsp]linear_proj_activ_scalar', 'lm/softmax/logits_ffn/[lsp]linear_proj_scalar', 'lm/total_aux_loss_scalar', 'lm/total_aux_loss_weight_scalar', 'lm/transformer/repeat/sub/x_la

In [318]:
# mesh =a
# paxml = b
div = np.abs(mesh - paxml)
div2 = (mesh - paxml)
result = []
print(f'mesh mean: {mesh.mean()}')
print(f'paxml mean: {paxml.mean()}')
print(f'paxml abs mean: {np.abs(paxml).mean()}')

print(f'div max: {div.max()}')
print(f'div abs mean: {div.mean()}')
print(f'div mean: {div2.mean()}')


ratio = div.mean() / paxml.mean()
print(f'div ratio: {ratio}')

result.append(div.max())
result.append(np.array(div.mean()))
result.append(np.array(div2.mean()))
result.append(mesh.mean())
result.append(paxml.mean())
result.append(np.abs(paxml).mean())

# result.append(ratio)


mesh mean: -2.069065809249878
paxml mean: 2.069187641143799
paxml abs mean: 2.0691874027252197
div max: 34.74753189086914
div abs mean: 4.138253688812256
div mean: -4.138253688812256
div ratio: 1.999941349029541


In [270]:
a = mesh[mesh > -100000]
b = paxml[paxml > -10000]

In [319]:
import pandas as pd

pd.DataFrame(result).T

,0,1,2,3,4,5
0,34.747532,4.1382537,-4.1382537,-2.0690658,2.0691876,2.069187


In [25]:
# 11.82132625579834
with jax.default_device(jax.devices("cpu")[0]):
    # paxml_data = pickle.load(open('/home/lishengping/projects/mesh_easy_jax/paxml_debug.pkl', 'rb'))
    paxml_data = pickle.load(open('/home/lishengping/projects/mesh_easy_jax/12_paxml_debug.pkl', 'rb'))
#     paxml_data = pickle.load(open('/home/lishengping/projects/paxml/debug.pkl', 'rb'))
    
# paxml_data.keys()

In [331]:
for i in range(paxml.shape[0]):
    a = (paxml[i] > 0).sum()
    b = (nmesh[i] > 0).sum()
    if a != b:
        print(i, a, b)

2 179 178
5 890 886
6 903 899
7 900 894


In [8]:
mesh_data['mask'][5].sum()

Array(890., dtype=float32)

In [372]:
tokenizer.decode(mesh_data['input_ids'][5][285])

'<0xEF>'

In [345]:
paxml[5].sum()

Array(4189.115, dtype=float32)

In [346]:
nmesh[5].sum()

Array(4188.8027, dtype=float32)

In [349]:
np.where(paxml[5] > 0)

(array([ 179,  180,  181,  182,  183,  184,  185,  186,  187,  274,  275,
         276,  277,  278,  279,  280,  281,  282,  283,  284,  285,  286,
         287,  288,  289,  290,  291,  292,  293,  294,  295,  296,  297,
         298,  299,  300,  301,  302,  303,  304,  305,  306,  307,  308,
         309,  310,  311,  312,  313,  314,  315,  316,  317,  318,  319,
         320,  321,  322,  323,  324,  325,  326,  327,  328,  329,  330,
         331,  332,  333,  334,  335,  336,  337,  338,  339,  340,  341,
         342,  343,  344,  345,  346,  347,  348,  349,  350,  351,  352,
         353,  354,  355,  356,  357,  358,  359,  360,  361,  362,  363,
         364,  365,  366,  367,  368,  369,  370,  371,  372,  373,  374,
         375,  376,  377,  378,  379,  380,  381,  382,  383,  384,  385,
         386,  387,  388,  389,  390,  457,  458,  459,  460,  461,  462,
         463,  464,  465,  466,  467,  468,  469,  470,  471,  472,  473,
         474,  475,  476,  477,  478, 

In [363]:
paxml[5][286]

Array(0.26146942, dtype=float32)

In [366]:
nmesh[5][285]

Array(-0., dtype=float32)

In [380]:
nmesh.sum()

Array(33883.023, dtype=float32)

In [382]:
nmesh.sum() / (nmesh > 0).sum()

Array(4.9696426, dtype=float32)

In [381]:
paxml.sum() / (paxml > 0).sum()

Array(4.9590244, dtype=float32)

In [357]:
np.where(nmesh[5] > 0)

(array([ 179,  180,  181,  182,  183,  184,  185,  186,  187,  274,  275,
         276,  277,  278,  279,  280,  281,  282,  283,  284,  286,  287,
         288,  289,  290,  291,  292,  293,  295,  296,  297,  298,  299,
         300,  301,  302,  303,  304,  306,  307,  308,  309,  310,  311,
         312,  313,  315,  316,  317,  318,  319,  320,  321,  322,  323,
         324,  325,  326,  327,  328,  329,  330,  331,  332,  333,  334,
         335,  336,  337,  338,  339,  340,  341,  342,  343,  344,  345,
         346,  347,  348,  349,  350,  351,  352,  353,  354,  355,  356,
         357,  358,  359,  360,  361,  362,  363,  364,  365,  366,  367,
         368,  369,  370,  371,  372,  373,  374,  375,  376,  377,  378,
         379,  380,  381,  382,  383,  384,  385,  386,  387,  388,  389,
         390,  457,  458,  459,  460,  461,  462,  463,  464,  465,  466,
         467,  468,  469,  470,  471,  472,  473,  474,  475,  476,  477,
         478,  479,  480,  481,  482, 

In [27]:
class_weights = mesh_data['mask']

In [30]:
per_example_xent = paxml_data['lm/[lsp]per_token_xent_scalar']

In [33]:
class_weights.shape

(8, 2047)

In [34]:
per_example_xent.shape

(8, 2047)

In [35]:
total_xent_batch = jnp.sum(
        per_example_xent * class_weights,
        dtype=jnp.float32, axis=-1,
    )
total_weight_batch = jnp.maximum(jnp.sum(class_weights, dtype=jnp.float32, axis=-1), 1e-10)
avg_xent = jnp.mean(total_xent_batch / total_weight_batch)

In [36]:
avg_xent

Array(5.0987988, dtype=float32)

In [26]:
paxml_data.keys()

dict_keys(['learning/applied_grad_norm_aggregate_scalar', 'learning/clipped_grad_norm_aggregate_scalar', 'learning/grad_scale_aggregate_scalar', 'learning/is_valid_step_aggregate_scalar', 'learning/lr_aggregate_scalar', 'learning/raw_grad_norm_aggregate_scalar', 'learning/var_norm_aggregate_scalar', 'lm/[lsp]input_ids_scalar', 'lm/[lsp]inputs_drop_scalar', 'lm/[lsp]inputs_scalar', 'lm/[lsp]last_output_norm_scalar', 'lm/[lsp]last_output_scalar', 'lm/[lsp]per_token_xent_scalar', 'lm/final_ln/[lsp]x_cheng_scalar', 'lm/final_ln/[lsp]x_eps_scalar', 'lm/final_ln/[lsp]x_mean_scalar', 'lm/final_ln/[lsp]x_rsqrt_scalar', 'lm/final_ln/[lsp]x_square_scalar', 'lm/num_unpadded_tokens_scalar', 'lm/ratio_unpadded_tokens_scalar', 'lm/softmax/logits_ffn/[lsp]linear_input_scalar', 'lm/softmax/logits_ffn/[lsp]linear_proj_activ_scalar', 'lm/softmax/logits_ffn/[lsp]linear_proj_scalar', 'lm/total_aux_loss_scalar', 'lm/total_aux_loss_weight_scalar', 'lm/transformer/repeat/sub/x_layers_0/[lsp]atten_output_0_sc

In [15]:
valid_text_length = mesh_data['valid_text_length']
valid_text_length

Array([1609.,  297.,  179., 1085.,  970.,  890.,  903.,  900.], dtype=float32)

In [16]:
token_log_prob =  mesh_data['loss']
token_log_prob

Array([[ 0.     ,  0.     ,  0.     , ..., -2.78125, -5.03125, -7.84375],
       [ 0.     ,  0.     ,  0.     , ...,  0.     ,  0.     ,  0.     ],
       [ 0.     ,  0.     ,  0.     , ...,  0.     ,  0.     ,  0.     ],
       ...,
       [ 0.     ,  0.     ,  0.     , ...,  0.     ,  0.     ,  0.     ],
       [ 0.     ,  0.     ,  0.     , ...,  0.     ,  0.     ,  0.     ],
       [ 0.     ,  0.     ,  0.     , ...,  0.     ,  0.     ,  0.     ]],      dtype=float32)

In [18]:
jnp.sum(token_log_prob, axis=-1)

Array([-7264.3804, -2025.3997,  -795.5207, -5461.2373, -5446.5776,
       -4188.8027, -3570.4795, -5130.628 ], dtype=float32)

In [19]:
token_log_prob[5].sum()

Array(-4188.8027, dtype=float32)

In [20]:
jnp.sum(token_log_prob, axis=-1) / valid_text_length

Array([-4.5148416, -6.819527 , -4.4442496, -5.0333986, -5.615029 ,
       -4.70652  , -3.954019 , -5.700698 ], dtype=float32)

In [22]:
jnp.sum(token_log_prob) / valid_text_length.sum()

Array(-4.958733, dtype=float32)

In [17]:
loss = -jnp.mean(jnp.sum(token_log_prob, axis=-1) / valid_text_length)
loss

Array(5.098535, dtype=float32)

In [14]:
mesh_data['mean_loss']

Array(5.0972733, dtype=float32)

In [355]:
mesh_index[20]

286

In [356]:
paxml_index[i]

285

In [354]:
mesh_index = np.where(nmesh[5] > 0)[0]
paxml_index = np.where(paxml[5] > 0)[0]
for i in range(len(mesh_index)):
    assert mesh_index[i] == paxml_index[i], print(i)

20


AssertionError: None

In [332]:
a

Array(900, dtype=int32)

In [334]:
paxml

Array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [335]:
nmesh[i]

Array([-0., -0., -0., ..., -0., -0., -0.], dtype=float32)

In [329]:
(paxml > 0).sum()

Array(6833, dtype=int32)

In [330]:
(nmesh > 0).sum()

Array(6818, dtype=int32)

In [326]:
(paxml[1] > 0).sum()

Array(297, dtype=int32)

In [328]:
nmesh = (-1 * mesh)
(nmesh[1] > 0).sum()

Array(297, dtype=int32)

In [376]:
paxml_data['lm/total_aux_loss_weight_scalar']

Array(0, dtype=bfloat16)

In [ ]:
atten_logits-0
div_max: 0.07999039 div_mean: 0.002445916, paxml-mean: 1.3579315
atten_logits-softmax-0
div_max: 0.01953125 div_mean: 3.3956853e-06, paxml-mean: 0.00048851874     
atten * value
div_max: 0.01953125 div_mean: 0.0015925519, paxml-mean: 0.47957897
ffn input:
div_max: 0.078125 div_mean: 0.0045422865, paxml-mean: 0.8717871
ffn norm:
div_max: 0.0703125 div_mean: 0.006677777, paxml-mean: 0.79627347
ffn1 gate:
div_max: 0.0625 div_mean: 0.0043873885, paxml-mean: 1.0804476

ffn1 activ:
div_max: 0.0625 div_mean: 0.0031632935, paxml-mean: 0.47032207
            
ffn1 out:
div_max: 0.0625 div_mean: 0.006678672, paxml-mean: 1.1028955
            
            
ffn2 out residual:
div_max: 0.25 div_mean: 0.019875739, paxml-mean: 2.0687337
            
ffn out:
div_max: 0.25 div_mean: 0.020937297, paxml-mean: 2.2158678

In [126]:
np.abs(b).mean()

2.068751

In [ ]:
去掉rmsnorm：
prob: 0.001953125
# post后
attn_out: 0.0009765625
ffn_input: 0.0009765625
ffnout resdiual: 9.1552734e-05
last_out: 0.0029296875
    
# grad lm head 0.016540527

In [47]:
step = 6002
path = 'gs://llm_base_models/baichuan_l2_debug'
with jax.default_device(jax.devices("cpu")[0]):
    item = {'params': orbax.checkpoint.Checkpointer(orbax.checkpoint.PyTreeCheckpointHandler())}
    mngr = orbax.checkpoint.CheckpointManager(path, item)
if step is None:
    step = mngr.latest_step()
w = mngr.restore(step)

I0000 00:00:1693794413.013146   16774 gcs_resource.cc:99] Using default AdmissionQueue with limit 32
I0000 00:00:1693794413.014709   26123 google_auth_provider.cc:179] Running on GCE, using service account 97048824446-compute@developer.gserviceaccount.com


In [3]:

step = 6002
path = 'gs://llm_base_models/baichuan_l2_debug'
with jax.default_device(jax.devices("cpu")[0]):
    item = {'params': orbax.checkpoint.Checkpointer(orbax.checkpoint.PyTreeCheckpointHandler())}
    mngr = orbax.checkpoint.CheckpointManager(path, item)
if step is None:
    step = mngr.latest_step()
w = mngr.restore(step)

I0000 00:00:1693874785.864579   15410 gcs_resource.cc:99] Using default AdmissionQueue with limit 32
I0000 00:00:1693874785.867925   16340 google_auth_provider.cc:179] Running on GCE, using service account 97048824446-compute@developer.gserviceaccount.com


In [ ]:
self.mngr.save(self.state['step'].item(), self.state)

In [4]:
jax.distributed.initialize()

In [6]:
path2 = 'gs://llm_base_models/baichuan_l2_debug/mesh_0905'
with jax.default_device(jax.devices("cpu")[0]):
    item2 = {'params': orbax.checkpoint.Checkpointer(orbax.checkpoint.PyTreeCheckpointHandler()),
            'step': orbax.checkpoint.AsyncCheckpointer(orbax.checkpoint.ArrayCheckpointHandler()),
}
    mngr2 = orbax.checkpoint.CheckpointManager(path2, item2)

In [ ]:
# from jax import random
# from jax import lax

# def _dropout(inputs, noise_shape):
#     if noise_shape is None:
#       noise_shape = inputs.shape
#     keep_prob = 0.9
#     prng_key = jax.random.PRNGKey(42)
#     assert keep_prob > 0.0
#     random_nums = keep_prob + jax.random.uniform(
#         prng_key, noise_shape, inputs.dtype, minval=0.0, maxval=1.0)
#     binary_mask = jnp.floor(random_nums)
#     return inputs * binary_mask / keep_prob, binary_mask

# def mesh_drop(inputs):
#     rate = 0.1
#     broadcast_dims = []
#     deterministic = False
#     keep_prob = 1.0 - rate
#     rng = jax.random.PRNGKey(42)
#     broadcast_shape = list(inputs.shape)
#     for dim in broadcast_dims:
#       broadcast_shape[dim] = 1
#     mask = random.bernoulli(rng, p=keep_prob, shape=broadcast_shape)
#     mask = jnp.broadcast_to(mask, inputs.shape)
#     return lax.select(mask, inputs / keep_prob, jnp.zeros_like(inputs)), mask

In [2]:
def _parse_function(example_proto): # https://zhuanlan.zhihu.com/p/552951305  # XD
    feature_desc = {"input_ids": tf.io.VarLenFeature(tf.int64), "labels": tf.io.VarLenFeature(tf.int64)}
    example = tf.io.parse_single_example(example_proto, feature_desc)
    for name in list(example.keys()):
        t = example[name]
        if t.dtype == tf.int64: t = tf.cast(t, dtype=tf.int32)
        example[name] = tf.sparse.to_dense(t, default_value=0)
        # example[name] = tf.sparse.to_dense(tf.sparse.reorder(t)) # mesh-transformer-jax
    return example


def shard(data, batch_size=None):  # XD
    return jax.tree_map(lambda x: x.numpy().reshape(batch_size + x.shape[1:]), data)  # mtj


def load_tfrecord_dataset(index_fname, batch_size, seq_len, restore_state=None, repeat=3, skip_step=0):  # XD
    tf.random.set_seed(1234)
    fnames = [index_fname] if index_fname.endswith('.tfrecords') else open(index_fname).read().splitlines()
    ds = tf.data.Dataset.from_tensor_slices(fnames)#.repeat()
    ds = ds.apply(tf.data.TFRecordDataset)
    # shard host data
    ds = ds.shard(jax.process_count(), jax.process_index())
    ds = ds.map(_parse_function, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.shuffle(buffer_size=10000) # 从文件中取buffer_size数据，然后打乱
    ds = ds.padded_batch(batch_size=np.prod(batch_size), 
                        padded_shapes={'input_ids': [seq_len], 'labels': [seq_len]},
                        padding_values={'input_ids': 0, 'labels': 0}, 
                        drop_remainder=True)
    ds = ds.prefetch(10)
    ds = ds.repeat(repeat)
    ds = ds.skip(skip_step)
    return map(lambda x: shard(x, batch_size=batch_size), iter(ds))

index_fname = "gs://jax_llm_data/data-baichuan/dreamily_translation_general.train.tfrecords"
batch_size = (1, 4)
seq_len = 2048
ds = load_tfrecord_dataset(index_fname, batch_size, seq_len, restore_state=None, repeat=3, skip_step=0) # XD
data_list = []
for i in range(200):
    d = next(ds)
    data_list.append(d['input_ids'])

for index, d in enumerate(data_list):
    id0 = [31278, 31188, 31178,    53, 31159,    53,    52,    73, 31488,
       28727, 31132, 21915, 12906,  6103, 31177,  3268, 10972, 31697,
          73,     5]
    id1 = d[0, 0, 30:30 + len(id0)].tolist()
    if id0 == id1:
        print(index)

NameError: name 'tf' is not defined